## Рефакторинг исходной таблицы

В этой тетрадке мы немножко играем с данными андийских языков.

Данные взяты [отсюда](https://docs.google.com/spreadsheets/d/1wLwgyR_YBa5yuKg8ozN4aLbi4wvRCEYkwX1Xhnryuuw/edit#gid=0). (Предоставлены проектом *Field NLP - morpology*) 

Наша задача распарсить данные и преобразовать их в онтологию. Нужно преобразовать словарные вхождения из таблицы в формат RDF и модель Ontolex lemon. 

На питон существует rdf либа:

In [5]:
# !pip install rdflib

Defaulting to user installation because normal site-packages is not writeable
  Using cached rdflib-5.0.0-py3-none-any.whl (231 kB)
  Using cached isodate-0.6.0-py2.py3-none-any.whl (45 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


RDF состоит из триплов, триплы это субъект-предикат-объект. Типа что - как_связано - с_чем. В либе rdflib вся эта сложная структура переведена в pythonic way -- у нас собирается что-то вроде контейнера\списка с 3-х элементными кортежами внутри. 

Вот пример из [документации](https://rdflib.readthedocs.io/en/stable/gettingstarted.html):

```
[
    (subject0, predicate0, object0),
    (subject1, predicate1, object1),
    ...
    (subjectN, predicateN, objectN)
 ]
```

Чтобы сразу не бросаться в огонь, я просто переведу саму таблицу с андийскими данными в pandas. А уже потом этот dataframe можно будет аккуратно переводить в rdf.

С гугл таблицами мне помог вот этот [туториал](https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c)

Но к сожалению, это работает везде кроме колаба. Видимо, колаб не умеет поднимать локальные сервера.

In [45]:
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle
from tqdm import tqdm

In [2]:
andi_sheet_id = "1wLwgyR_YBa5yuKg8ozN4aLbi4wvRCEYkwX1Xhnryuuw"

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# here enter the id of your google sheet
SAMPLE_SPREADSHEET_ID_input = andi_sheet_id
SAMPLE_RANGE_NAME = 'A1:S68794'

def main():
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)  # here enter the name of your downloaded JSON file
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                                range=SAMPLE_RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input:
        print('No data found.')

main()

df = pd.DataFrame(values_input[1:], columns=values_input[0])

In [3]:
df.head()

,,id_word,id_meaning,id,lemma,ipa,morphology,bor,annotator,pos,concepticon,meaning_ru,definition,glottocode,reference,comments,borrowing_source_word,borrowing_source_comment,"45,42%"
0,ghod1238,1,1,1,АБÁДАЛIИ,,,,George Moroz,adv,1656,никогда,нареч. 1) никогда; абадалIи йаиччи ден биттиха...,ghod1238,Saidova 2006,,,,3049
1,,1,2,2,АБÁДАЛIИ,,,,George Moroz,adv,,ни за что,нареч. 1) никогда; абадалIи йаиччи ден биттиха...,ghod1238,Saidova 2006,,,,7932
2,,2,1,3,АБÁДИЛ-АБÁДАЛIИ,,,,George Moroz,adv,,см. АБÁДАЛIИ,"то же, что абáдалIи",ghod1238,Saidova 2006,,,,10351
3,,3,1,4,АБАЗИ́НА/В,,(-щуб),1,George Moroz,noun,,абазинец,I (-щуб) абазинец,ghod1238,Saidova 2006,,,,1391
4,,4,1,5,АБАЗИ́НА/Й,,(-лълъилIи),1,George Moroz,noun,,абазинец,II (-лълъилIи) абазинка,ghod1238,Saidova 2006,,,,4124


Делаем кокой-то RDF (пока не знаем какой, просто делаем, чтобы знать, что мы такое умеем)

In [6]:
import rdflib
from rdflib.namespace import RDF, RDFS, OWL, DC, DCTERMS
from rdflib.term import URIRef, Literal

In [54]:
g = rdflib.Graph(identifier='http://example.org/ontolex/andi#')  # пока рандомная ссылка

In [55]:
andi = rdflib.Namespace('http://example.org/ontolex/andi#')  # Корневой Namespace с рандомной ссылкой

ontolex = rdflib.Namespace('http://www.w3.org/ns/lemon/ontolex#')  # основная модель
lexinfo = rdflib.Namespace('http://www.lexinfo.net/ontology/2.0/lexinfo#')  # обозначение морфологических тегов
lime = rdflib.Namespace('http://www.w3.org/ns/lemon/lime#')
#synsem = rdflib.Namespace('http://www.w3.org/ns/lemon/synsem#')  # синтаксис
#decomp = rdflib.Namespace('http://www.w3.org/ns/lemon/decomp#')  # синтаксис
vartrans = rdflib.Namespace('http://www.w3.org/ns/lemon/vartrans#')  # для обозначения переводов и вариантов слов

In [56]:
g.bind('ontolex', ontolex)
g.bind('lexinfo', lexinfo)
g.bind('dct', DCTERMS)  # Это для того, чтобы названия ISO языков грузились
g.bind('rdfs', RDFS)
g.bind('lime', lime)  # ??
#g.bind('synsem', synsem)
#g.bind('decomp', decomp)
g.bind('vartrans', vartrans)

lexicon_andi = URIRef(andi)

In [57]:
# Надо вот тут добавить все типы триплов, чтобы они добавились в префиксы ниже
g.set((lexicon_andi, RDF.type, lime.Lexicon))
g.set((lexicon_andi, ontolex.language, Literal("ru, ani")))
g.set((lexicon_andi, RDFS.comment, Literal("lexicon-information_name:Andi-Russian", lang="en")))

#lime:Lexicon_Kadag1884_bbl a ontolex:Lexicon.
#lime:Lexicon_Kadag1884_bbl ontolex:language "bbl", "ka", "ru".
#lime:Lexicon_Kadag1884_bbl rdfs:comment "lexicon-information_name:Kadagidze - Batsbi-Georgian-Russian"@en.

#g.set((apics_doc, ligt.hasUtterances, apics.examples))
#g.set((apics.examples, RDF.type, ligt.InterlinearCollection))
#Probably it should also be a `dc:bibliographicCitation`
#g.set((apics.examples, RDFS.comment, Literal(apics_dataset.properties['dc:bibliographicCitation '], lang="en")))

In [58]:
# сделала пустой turtle, прикольно
print(g.serialize(format='turtle').decode('utf-8'))

@prefix lime: <http://www.w3.org/ns/lemon/lime#> .
@prefix ontolex: <http://www.w3.org/ns/lemon/ontolex#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://example.org/ontolex/andi#> a lime:Lexicon ;
    rdfs:comment "lexicon-information_name:Andi-Russian"@en ;
    ontolex:language "ru, ani" .




In [59]:
for lemma in tqdm(df['lemma '][:1000]):
    if ' ' in lemma:
        lemma = lemma.replace(' ', '_')
        
    g.add((Literal(lemma, lang='ani'), RDF.type, ontolex.LexicalEntry))
    g.add((Literal(lemma, lang='ani'), DCTERMS.language, URIRef('http://id.loc.gov/vocabulary/iso639-3/ani')))
    g.add((Literal(lemma, lang='ani'), ontolex.sense, Literal(lemma + '_sense')))
    
    g.add((Literal(lemma, lang='ani'), lexinfo.partOfSpeech, lexinfo.noun))
    
    # у некоторых слов есть несколько переводов, это обозначено, как новые строки таблицы
    # пока чтобы не париться я беру только один перевод
    translation = df[df['lemma '] == lemma]['meaning_ru']
    if not translation.empty:
        translation = translation.values[0]
    else:
        translation = '_'
    
    if ' ' in translation:
        translation = translation.replace(' ', '_')
        
    g.add((Literal(lemma + '_sense'), ontolex.reference, Literal(translation + '_concept')))
    # :surrogate_mother_sense ontolex:reference ontology:SurrogateMother.
    
    g.add((Literal(translation, lang='ru'), RDF.type, ontolex.LexicalEntry))
    g.add((Literal(translation, lang='ru'), DCTERMS.language, URIRef('http://id.loc.gov/vocabulary/iso639-2/rus')))
    g.add((Literal(translation, lang='ru'), ontolex.sense, Literal(translation + 'sense')))

    g.add((Literal(translation + '_sense'), ontolex.reference, Literal(translation + '_concept')))
    # :leihmutter_sense ontolex:reference ontology:SurrogateMother.
    

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 68.54it/s]


In [60]:
print(g.serialize(format='turtle').decode('utf-8'))

@prefix dct: <http://purl.org/dc/terms/> .
@prefix lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#> .
@prefix lime: <http://www.w3.org/ns/lemon/lime#> .
@prefix ontolex: <http://www.w3.org/ns/lemon/ontolex#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://example.org/ontolex/andi#> a lime:Lexicon ;
    rdfs:comment "lexicon-information_name:Andi-Russian"@en ;
    ontolex:language "ru, ani" .

"__sense" ontolex:reference "__concept" .

"Аллах_sense" ontolex:reference "Аллах_concept" .

"абазинец_sense" ontolex:reference "абазинец_concept" .

"абазины_sense" ontolex:reference "абазины_concept" .

"абхазец_sense" ontolex:reference "абхазец_concept" .

"абхазка_sense" ontolex:reference "абхазка_concept" .

"абхазы_sense" ontolex:reference "абхазы_concept" .

"аванс_sense" ontolex:reference "аванс_concept" .

"аварский_sense" ontolex:reference "аварский_concept" .

"август_sense" ontolex:reference "август_concept" .

"автобус_sense" ontolex:reference "автобус_conce